In [11]:
'''
Created on Oct 16, 2015

@author: c3h3
'''

import requests
from pyquery import PyQuery
import pandas as pd
import urllib
from datetime import datetime


def getData(n_past_data = 100):
    
    list_df = get_headline_list(n_past_data)
    
    data = []
    
    for url in list_df["url"][list_df["url"].notnull()]:
        try:
            d = get_news_data(url)
            data.append(d)
        except Exception as e:
            print "e = ",e
            
    news_df = pd.DataFrame(data)
    
    return {"list_df":list_df, "news_df":news_df} 


def get_headline_list(n_past_data = 100): 

    assert isinstance(n_past_data, int)
    s = requests.Session()
    res = s.get("http://www.marketwatch.com/newsviewer")
    S = PyQuery(res.text)
    data = S("li").map(lambda i,e: {"id": PyQuery(e).attr("id"),
                                    "timestamp": PyQuery(e).attr("timestamp"),
                                    "title": PyQuery(e)(".nv-text-cont").text(),
                                    "details": PyQuery(e)(".nv-details").outerHtml(),
                                    "url": PyQuery(e)(".nv-text-cont a").attr("href"),
                                   })
    
    if n_past_data > 0:
        postData = dict(map(lambda xx: (xx.split("=")[0],urllib.unquote(xx.split("=")[1])) ,
                            "topstories=true&rtheadlines=true&pulse=true&commentary=true&video=true&premium=true&blogs=true&topic=All+Topics&docId=1187576694&timestamp=10%2F15%2F2015+10%3A01%3A32+AM&pullCount=10".split("&")))
    
        postData["timestamp"] = data[-1]["timestamp"]
        postData["docId"] = data[-1]["id"]
        postData["topic"] = postData["topic"].replace("+"," ")
        postData["pullCount"] = n_past_data
        
        pastRes = s.post("http://www.marketwatch.com/newsviewer/mktwheadlines",data=postData)
        pastS = PyQuery(pastRes.text)
        pastData = pastS("li").map(lambda i,e: {"id": PyQuery(e).attr("id"),
                                                "timestamp": PyQuery(e).attr("timestamp"),
                                                "title": PyQuery(e)(".nv-text-cont").text(),
                                                "details": PyQuery(e)(".nv-details").outerHtml(),
                                                "url": PyQuery(e)(".nv-text-cont a").attr("href"),
                                               })
        
    data.extend(pastData)
    df = pd.DataFrame(data)
    df = df[df["timestamp"].notnull()]
    df["url"] = df["url"].map(lambda url: "http://www.marketwatch.com"+url if url else url)
    df["time"] = df["timestamp"].map(lambda tt: datetime.strptime(tt,"%m/%d/%Y %I:%M:%S %p") if tt else tt)
    
    df.sort("time",ascending=False)
    
    return df

def get_news_data(url):
    print "url = ",url
    res = requests.get(url)
    S = PyQuery(res.text)
    
    article_data = {}
    
    article_data["url"] = url
    article_data["headline"] = S("#article-headline").text()
    article_data["published-timestamp"] = S("#published-timestamp").text()
    article_data["body"] = S("#article-body").outerHtml()
    article_data["related-articles"] = S("related-articles").outerHtml()
    
    return article_data


In [12]:
d = getData()

url =  http://www.marketwatch.com/story/most-stock-market-gains-are-made-when-youre-asleep-2015-10-14
url =  http://www.marketwatch.com/story/ford-recalling-kuga-model-suvs-in-china-for-fuel-leakage-2015-10-16
url =  http://www.marketwatch.com/story/ge-profit-revenue-top-expectations-2015-10-16-64854957
url =  http://www.marketwatch.com/story/5-telling-charts-from-this-surprising-week-for-global-markets-2015-10-16
url =  http://www.marketwatch.com/story/the-new-normal-everything-stinks-2015-10-16
url =  http://www.marketwatch.com/story/hsbc-may-need-more-time-on-headquarters-decision-2015-10-16
url =  http://www.marketwatch.com/story/us-investigation-into-volkswagen-cheating-allegations-widens-2015-10-16
url =  http://www.marketwatch.com/story/wall-street-looks-for-signs-sp-500-can-hang-onto-2000-2015-10-16
url =  http://www.marketwatch.com/story/whats-an-arm-and-a-leg-worth-to-you-2015-10-16
url =  http://www.marketwatch.com/story/uber-wins-london-black-cab-battle-in-uk-court-reports-

In [14]:
d["news_df"]

,body,headline,published-timestamp,related-articles,url
0,"<div id=""article-body"" itemprop=""articleBody"">...",Most stock market gains are made when you’re a...,"Published: Oct 16, 2015 6:57 a.m. ET",None,http://www.marketwatch.com/story/most-stock-ma...
1,"<div id=""article-body"" itemprop=""articleBody"">...",Ford recalling Kuga-model SUVs in China for fu...,"Published: Oct 16, 2015 6:52 a.m. ET",None,http://www.marketwatch.com/story/ford-recallin...
2,"<div id=""article-body"" itemprop=""articleBody"">...","GE profit, revenue top expectations","Published: Oct 16, 2015 6:49 a.m. ET",None,http://www.marketwatch.com/story/ge-profit-rev...
3,"<div id=""article-body"" itemprop=""articleBody"">...",5 telling charts from this surprising week for...,"Published: Oct 16, 2015 6:24 a.m. ET",None,http://www.marketwatch.com/story/5-telling-cha...
4,"<div id=""article-body"" itemprop=""articleBody"">...",The ‘new normal’ for markets: Everything stinks,"Published: Oct 16, 2015 6:23 a.m. ET",None,http://www.marketwatch.com/story/the-new-norma...
5,"<div id=""article-body"" itemprop=""articleBody"">...",HSBC may need more time on headquarters decision,"Published: Oct 16, 2015 6:09 a.m. ET",None,http://www.marketwatch.com/story/hsbc-may-need...
6,"<div id=""article-body"" itemprop=""articleBody"">...",U.S. investigation into Volkswagen cheating al...,"Published: Oct 16, 2015 6:04 a.m. ET",None,http://www.marketwatch.com/story/us-investigat...
7,"<div id=""article-body"" itemprop=""articleBody"">...",Wall Street looks for signs S&P 500 can hang o...,"Published: Oct 16, 2015 6:03 a.m. ET",None,http://www.marketwatch.com/story/wall-street-l...
8,"<div id=""article-body"" itemprop=""articleBody"">...",What’s an arm and a leg worth to you?,"Published: Oct 16, 2015 6:00 a.m. ET",None,http://www.marketwatch.com/story/whats-an-arm-...
9,"<div id=""article-body"" itemprop=""articleBody"">...",Uber wins London black-cab battle in U.K. cour...,"Published: Oct 16, 2015 5:49 a.m. ET",None,http://www.marketwatch.com/story/uber-wins-lon...
